# Imports

In [16]:
import pandas as pd; import os; from analysis_helpers import *; import warnings
warnings.filterwarnings('ignore')

# Some Functions

In [25]:
def list_logs(data_dir):
    '''
    input : path to participant data directory  (str)
    output: participant's full behavioral log   (df)
    '''
            
    participant = []
    for f in os.listdir(data_dir):

        if f !='--1.log' and ('.log' in f):

            with open(data_dir+f) as file:

                    lines = file.read().splitlines()
                    lines = [[lines[x]] for x in range(0, len(lines))]
                    #lines.insert(0,['DATA'])

                    log_file = pd.DataFrame(lines).reset_index()

                    log_file['Subject'] = sub_dir.split('_')[0]
                    log_file['Run'] = int(f.split('.')[-2][-1])
                    log_file['TIME'],log_file['WARNING'],log_file['MESSAGE'] = log_file[0].str.split('\t', 4).str
                    log_file = log_file.fillna('NAN VALUE')
                    log_file['TIME'] = log_file['TIME'].str.replace(' ','')
                    log_file['TIME'] = log_file['TIME'].astype(float, errors='ignore')

                    a = log_file[log_file[0].str.contains('current')][0].str.split(' ', expand=True)
                    log_file['TIME'] = log_file[['TIME']].applymap(lambda x: np.nan if isinstance(x, str) else float(x))
                    #log_file['TIME'] = log_file['TIME'] + float(a[4]) - float(a[0])

#                             utc = log_file[0].str.contains('current')['MESSAGE'].split(' ')[-1]
#                             psycho = log_file[0].str.contains('current')['TIME']

                    participant.append(log_file)

    participant = pd.concat(participant)
    return(participant)

# Data Compile
Organize behavioral data from each participant into one dataframe for the whole experiment.

In [126]:
# Compile data from all exps
# add attention labels to memory stim
# correct timings for group1 - safest estimate
# check and add gaze data
# output dataframe with attn labels, timing, and gaze data

# run behavioral checks and further gaze checks

a,b = '../sustained_attention_experiment/data/','../variable_attention_experiment/data/'
data_dirs = [a+'group1', a+'group2', b+'group1', b+'group2']
filenames = [x+'_partial.csv' for x in data_dirs]

sub_count = 0

for data,file in zip(data_dirs, filenames):
    
    sub_list = []
    
    # for each subject in this directory
    for sub_dir in os.listdir(data):
        
        if sub_dir != '.DS_Store' and sub_dir != 'README.md':
            
            # add attention labels to memory stim 
            subject = add_level(sum_pd(data + '/' + sub_dir))
            
            # TIMING CORRRECTIONS (for gaze)
            if data[-1] == '1':
               
                # github Issue #83, Check B: times in behavioral csv's (Group1) imprecise by fractions of a second
                # correct for those timing issues (for gaze analysis), using safest estimates 
                
                # SUSTAINED: cued composite starts .0178 seconds earlier; VAR : .0167 earlier 
                # SUSTAINED: composites disappear 0.0322 secs later; VARIABLE: .0359 secs 
                subject.loc[subject['Trial Type']=='Presentation','Stimulus Onset'] = subject[subject['Trial Type']=='Presentation']['Stimulus Onset'] - .0178
                subject.loc[subject['Trial Type']=='Presentation','Stimulus End']   = subject[subject['Trial Type']=='Presentation']['Stimulus End']   + .0359
                  
                # SUST: mem images display 0.0179 earlier; VAR: .0142
                # SUST: mem images disappear .259 secs later; VAR: .137
                subject.loc[subject['Trial Type']=='Memory','Stimulus Onset'] = subject[subject['Trial Type']=='Memory']['Stimulus Onset'] - .0179
                subject.loc[subject['Trial Type']=='Memory','Stimulus End']   = subject[subject['Trial Type']=='Memory']['Stimulus End']   +.259
                
                # NOTE: no correction for button press differences, as they averaged .001 seconds or less

                if data == '../variable_attention_experiment/data/group1':
                    # EXP 2 cue corrections ---------------
                    # github Issue #83, Check E: some Group1, Var Exp valid cues marked as invalid
                    
                    # correct cue labels Var Exp, Group 1
                    for run in subject['Run'].unique():
                        first_cue = list(subject[(subject['Run']==run) & (subject['Trial Type']=='Presentation')]['Cued Side'])
                        if subject[(subject['Run']==run) & (subject['Cue Validity']==0) & (subject['Cued Side'] != first_cue[0])].shape[0]>0:
                            subject.loc[(subject['Run']==run) & (subject['Cue Validity']==0) & (subject['Cued Side'] != first_cue[0]), 'Cue Validity'] = 1
            
            if data[-1] == '2':
                
                # add ON and OFF stim times for group 2
                subject_log = list_logs(data + '/' + sub_dir+'/')
                subject_log['Subject'] = pd.to_numeric(subject_log['Subject'])
                subject_log = subject_log.sort_values(by=['Subject','Run','TIME'])
                subject = subject.sort_values(by=['Subject','Run'])
                
                # now, extract desired stim on and off times from log files
                composite_onsets  = subject_log[subject_log[0].str.contains('COMPOSITES ON')]
                composite_offsets = subject_log[subject_log[0].str.contains('COMPOSITES OFF')]
                memory_onsets  = list(subject_log[ (subject_log[0].str.contains('MEMORY ON')) & (subject_log[0].str.contains('FLIP')) ]['TIME'])
                memory_offsets = list(subject_log[ (subject_log[0].str.contains('MEMORY OFF')) & (subject_log[0].str.contains('FLIP')) ]['TIME'])
                # attention_on = composite_offsets 
                
                # add ON and OFF stim times for group 2
                subject.loc[subject['Trial Type']=='Presentation', 'Stimulus Onset'] = list(composite_onsets['TIME'])
                subject.loc[subject['Trial Type']=='Presentation', 'Stimulus End'  ] = list(composite_offsets['TIME'])
                subject.loc[subject['Trial Type']=='Presentation','Attention Response Time (s)'] = subject[subject['Trial Type']=='Presentation']['Attention Response Time (s)'] - subject[subject['Trial Type']=='Presentation']['Stimulus End']
                subject.loc[subject['Trial Type']=='Memory', 'Stimulus Onset'] = memory_onsets
                subject.loc[subject['Trial Type']=='Memory', 'Stimulus End'  ] = memory_offsets
                
                # Convert all times to be eyetribe compatible
                for r in subject['Run'].unique():
                    time = float(subject_log[subject_log['Run']==r].loc[subject_log[subject_log['Run']==r][0].str.contains('urrent time')]['TIME'])
                    curr_string = subject_log[subject_log['Run']==r].loc[subject_log[subject_log['Run']==r][0].str.contains('urrent time')][0].str.split(' ')
                    curr_time = float(list(curr_string)[0][-1])
                    diff = curr_time - time

                    # convert times for each run
                    subject.loc[subject['Run']==r, 'Stimulus Onset'] = subject.loc[subject['Run']==r, 'Stimulus Onset'] + diff
                    subject.loc[subject['Run']==r, 'Stimulus End']   = subject.loc[subject['Run']==r, 'Stimulus End'] + diff

            # GROUP 2 -- pull attention RT's from log file
            
            
            
            # Add gaze data (internal)
            
            
            # Give every subj unique ID, label group & experiment
            subject['UniqueID'] = sub_count
            subject['Group'] = int(data[-1])
            subject['Experiment'] = data[2:10]
            
            print(sub_count)
            sub_count += 1
            sub_list.append(subject)
            
    exp_raw = pd.concat(sub_list)
    exp_raw.to_csv(file)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
